In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import pandas as pd
from llm_feature_selector import LLMFeatureSelector
import numpy as np

In [3]:
current = pd.read_csv('CurrentVoltage.csv', index_col='DeviceTimeStamp')
overview = pd.read_csv('Overview.csv', index_col='DeviceTimeStamp')
power = pd.read_csv('Power.csv', index_col='DeviceTimeStamp')
power_factor = pd.read_csv('PowerFactor.csv', index_col='DeviceTimeStamp')
total_power = pd.read_csv('TotalPower.csv', index_col='DeviceTimeStamp')

In [4]:
current.index = pd.to_datetime(current.index)
overview.index = pd.to_datetime(overview.index)
power.index = pd.to_datetime(power.index)
power_factor.index = pd.to_datetime(power_factor.index)
total_power.index = pd.to_datetime(total_power.index)

In [5]:
transformer = current
for df in [overview, power, power_factor, total_power]:
    transformer = pd.merge(transformer, df, on='DeviceTimeStamp')
transformer = transformer.reset_index()

In [7]:
transformer['MOG_A'] = transformer['MOG_A'].shift(1)

In [6]:
transformer.columns

Index(['DeviceTimeStamp', 'VL1', 'VL2', 'VL3', 'IL1', 'IL2', 'IL3', 'VL12',
       'VL23', 'VL31', 'INUT', 'OTI', 'WTI', 'ATI', 'OLI', 'OTI_A', 'OTI_T',
       'MOG_A', 'WL1', 'WL2', 'WL3', 'VAL1', 'VAL2', 'VAL3', 'RVAL1', 'RVAL2',
       'RVAL3', 'PFL1', 'PFL2', 'PFL3', 'Avg_PF', 'Sum_PF', 'FRQ', 'THDVL1',
       'THDVL2', 'THDVL3', 'THDIL1', 'THDIL2', 'THDIL3', 'MDIL1', 'MDIL2',
       'MDIL3', 'KWH', 'KWH_I', 'KVARH', 'KW', 'KVA', 'KVAR', 'MPD', 'MKVAD'],
      dtype='object')

In [7]:
# feature_descriptions = {
#     'VL1': 'Phase Line 1',
#     'VL2': 'Phase Line 2',
#     'VL3': 'Phase Line 3',
#     'IL1': 'Current Line 1',
#     'IL2': 'Current Line 2',
#     'IL3': 'Current Line 3',
#     'VL12': 'Voltage line 1 2',
#     'VL23': 'Voltage line 2 3',
#     'VL31': 'Voltage line 3 1',
#     'INUT': 'Neutral Current',
#     'OTI': 'Oil Temperature Indicator',
#     'WTI': 'Winding Temperature Indicator',
#     'ATI': 'Ambient Temperature Indicator',
#     'OLI': 'Oil Level Indicator',
#     'OTI_A': 'Oil Temperature Indicator Alarm',
# }

In [10]:
# model_name = 'microsoft/phi-4'
model_name = 'meta-llama/Llama-3.3-70B-Instruct'
prompt_template = 'Select the variables from the list that are most relevant for predicting <target_variable>. ' +\
                  'Provide the variables sorted starting with the one with the highest priority. ' +\
                  'All variables: <all_variables>\n' + \
                  '```json\n{"reasoning": "<your reasoning>", "selected_variables": ["variable 1", "variable 2", ..., "variable n"]}\n```'
all_cols = transformer.drop(['DeviceTimeStamp', 'MOG_A'], axis=1).columns.tolist()

In [12]:
prompt = prompt_template.replace('<all_variables>', ', '.join(all_cols))
prompt = prompt.replace('<target_variable>', 'magnetic oil gauge fault in electrical transformers')
print(prompt)

Select the variables from the list that are most relevant for predicting magnetic oil gauge fault in electrical transformers. Provide the variables sorted starting with the one with the highest priority. All variables: VL1, VL2, VL3, IL1, IL2, IL3, VL12, VL23, VL31, INUT, OTI, WTI, ATI, OLI, OTI_A, OTI_T, WL1, WL2, WL3, VAL1, VAL2, VAL3, RVAL1, RVAL2, RVAL3, PFL1, PFL2, PFL3, Avg_PF, Sum_PF, FRQ, THDVL1, THDVL2, THDVL3, THDIL1, THDIL2, THDIL3, MDIL1, MDIL2, MDIL3, KWH, KWH_I, KVARH, KW, KVA, KVAR, MPD, MKVAD
```json
{"reasoning": "<your reasoning>", "selected_variables": ["variable 1", "variable 2", ..., "variable n"]}
```


In [24]:
feat_sel = LLMFeatureSelector(model_name=model_name,
                              feature_names=all_cols,
                              target_variable='magnetic oil gauge fault',
                              prompt_template=prompt_template,
                              topk=3
                             )
output = feat_sel.fit(transformer[all_cols])
output.shape

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{"selected_variables": ["OTI", "OLI", "INUT", "KWH", "KW", "KVA", "KVAR", "MDIL1", "MDIL2", "MDIL3", "THDIL1", "THDIL2", "THDIL3", "THDVL1", "THDVL2", "THDVL3", "VL1", "VL2", "VL3", "IL1", "IL2", "IL3"]}


(3, 48)

In [26]:
sorted_corrs = transformer.drop('DeviceTimeStamp', axis=1).corr()['MOG_A'].abs().sort_values(ascending=False).to_frame()
sorted_corrs['idx'] = np.arange(len(sorted_corrs))

## We manually took the prompt and fed it to ChatGPT

In [9]:
chatgpt_response = {
  "reasoning": "Magnetic oil gauge faults in electrical transformers are most often linked to issues in oil temperature, pressure, or flow, which in turn are influenced by thermal and electrical stress on the transformer. Variables directly indicating oil and winding temperatures (OTI, WTI, OLI) are critical, as they reflect the thermal state of the transformer and can signal overheating or abnormal cooling—both major contributors to oil-related faults. Load currents (IL1, IL2, IL3) are also important, as high current increases heat generation. Power measurements (KW, KVA), power factor (PFL1-3, Avg_PF), and harmonics (THDIL1-3) can indicate operational stress and efficiency, which affect oil performance. Derived temperature indicators (OTI_A, OTI_T) may provide additional granularity. Voltage metrics and frequency are less directly linked to oil faults unless they're highly unstable, and derived values such as MPD or MKVAD are likely less informative unless defined otherwise. Thus, the selection prioritizes direct indicators of thermal and electrical stress.",
  "selected_variables": [
    "OTI",
    "WTI",
    "OLI",
    "OTI_A",
    "OTI_T",
    "IL1",
    "IL2",
    "IL3",
    "KW",
    "KVA",
    "PFL1",
    "PFL2",
    "PFL3",
    "Avg_PF",
    "THDIL1",
    "THDIL2",
    "THDIL3",
    "MDIL1",
    "MDIL2",
    "MDIL3",
    "KWH"
  ]
}

In [39]:
[f'{round(sorted_corrs.loc[sorted_corrs.index==rec].iloc[0].MOG_A * 100, 2)} ({int(sorted_corrs.loc[sorted_corrs.index==rec].iloc[0].idx)})' for rec in chatgpt_response['selected_variables']]

['7.84 (38)',
 '12.52 (33)',
 '20.33 (24)',
 '0.25 (47)',
 '0.31 (46)',
 '28.09 (15)',
 '35.13 (3)',
 '27.24 (17)',
 '29.34 (14)',
 '29.43 (13)',
 '4.72 (40)',
 '7.66 (39)',
 '22.26 (23)',
 '13.91 (31)',
 '1.01 (44)',
 '10.86 (36)',
 '3.99 (41)',
 '27.01 (19)',
 '35.56 (2)',
 '26.09 (21)',
 '19.47 (28)']